<a href="https://colab.research.google.com/github/Mansi-Nayak/Agents_Langchain/blob/main/agents_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
This script demonstrates how to build a lightweight, open-source, CPU-friendly toolchain using
LangChain and Hugging Face models to answer a multi-step natural language question.

1. It loads a CPU-compatible text generation model (`flan-t5-base`) via Hugging Face Transformers.
2. Uses `DuckDuckGoSearchRun` from LangChain Community to fetch factual answers from the web.
3. Defines a custom tool `get_weather_data` to fetch current weather info using the Weatherstack API.
4. It searches for the capital city of Madhya Pradesh using DuckDuckGo.
5. Extracts the capital city name using regular expressions (fallback to "Bhopal" if parsing fails).
6. Calls the weather tool to get current temperature, humidity, and conditions for that city.
7. Constructs a prompt that includes both the city and weather for LLM summarization.
8. Chains the prompt and LLM together using LangChain's `RunnableSequence`.
9. Invokes the LLM to generate a human-readable response based on the collected data.
10. Prints the LLM-generated response to the console.
11. If the model fails to mention weather, it prints a fallback weather string directly.
12. This approach avoids the ReAct agent format, making it more compatible with small models.
13. Useful for building simple assistants or tool-using bots on CPU environments (like Colab).
14. Can be extended with more tools (e.g., news, currency converters, etc.) if needed.
15. Demonstrates prompt engineering and tool integration with minimal dependencies.
"""

In [1]:
!pip install -q langchain langchain-community langchain-core duckduckgo-search transformers accelerate sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.9/438.9 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 67.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/1

In [16]:
import requests
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.tools import tool
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub
from langchain_community.llms import HuggingFacePipeline
from transformers import pipeline

In [30]:
hf_pipeline = pipeline(
    "text2text-generation",
    model="google/flan-t5-large",
    max_new_tokens=256
)

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cpu


In [31]:
# Wrap it with LangChain-compatible LLM
from langchain_core.language_models import LLM
llm = HuggingFacePipeline(pipeline=hf_pipeline)

In [32]:
# Search tool using DuckDuckGo
search_tool = DuckDuckGoSearchRun()

In [33]:
@tool
def get_weather_data(city: str) -> str:
    """Fetch current weather data for a given city using Weatherstack API."""
    url = f'https://api.weatherstack.com/current?access_key=4d1d8ae207a8c845a52df8a67bf3623e&query={city}'
    response = requests.get(url)
    return str(response.json().get("current", response.json()))

In [34]:
# 3. Pull official ReAct prompt (includes all needed vars)
prompt = hub.pull("hwchase17/react")

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [35]:
# 4. Create agent with tools
tools = [search_tool, get_weather_data]
agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)

In [36]:
# 5. Create executor with error handling
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True, max_iterations=10, max_execution_time=60)

In [37]:
# 6. Invoke agent
response = agent_executor.invoke({
    "input": "Find the capital of Madhya Pradesh, then find its current weather condition"
})



> Entering new AgentExecutor chain...
[duckduckgo_search, get_weather_data(state) -> stateInvalid Format: Missing 'Action:' after 'Thought:'[duckduckgo_search, get_weather_data(state) -> stateInvalid Format: Missing 'Action:' after 'Thought:'[duckduckgo_search, get_weather_data(state) -> stateInvalid Format: Missing 'Action:' after 'Thought:'[duckduckgo_search, get_weather_data(state) -> stateInvalid Format: Missing 'Action:' after 'Thought:'[duckduckgo_search, get_weather_data(state) -> stateInvalid Format: Missing 'Action:' after 'Thought:'

> Finished chain.


In [38]:
# 7. Show final answer
print(response["output"])

Agent stopped due to iteration limit or time limit.


In [49]:
from transformers import pipeline
from langchain_community.llms import HuggingFacePipeline
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.tools import tool
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableSequence
import requests
import re

In [50]:
hf_pipeline = pipeline("text2text-generation", model="google/flan-t5-base", max_new_tokens=256)
llm = HuggingFacePipeline(pipeline=hf_pipeline)

Device set to use cpu


In [51]:
search = DuckDuckGoSearchRun()


In [73]:
@tool
def get_weather_data(city: str) -> str:
    """Get weather using OpenWeatherMap API."""
    api_key = "e818196642428071376950788ce7d2e4"
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"
    res = requests.get(url).json()
    if res.get("main"):
        desc = res["weather"][0]["description"]
        temp = res["main"]["temp"]
        humidity = res["main"]["humidity"]
        return f"{desc.capitalize()}, Temp: {temp}°C, Humidity: {humidity}%"
    return "Weather data not available."


In [74]:
# Step 1: Find capital
capital_info = search.invoke("Capital of Madhya Pradesh")
match = re.search(r'Capital of Madhya Pradesh.*?\s+is\s+([A-Z][a-z]+)', capital_info)
city = match.group(1) if match else "Bhopal"# Step 1: Search capital of Madhya Pradesh
capital = search.invoke("Capital of Madhya Pradesh")

In [68]:
# Step 2: Get weather
weather = get_weather_data.invoke(city)

In [69]:
# Step 3: Use prompt to summarize
prompt = PromptTemplate.from_template("""
You are a helpful assistant.

Question: What is the capital of Madhya Pradesh and what is the weather like there?

Answer:
The capital of Madhya Pradesh is {city}.
The current weather in {city} is: {weather}.
""")

In [70]:

chain = prompt | llm
response = chain.invoke({"city": city, "weather": weather})

In [71]:
# Step 4: Print both model output and fallback if needed
print("\n LLM Response:\n", response)


📌 LLM Response:
 Bhopal


In [72]:

if "weather" not in response.lower():
    print("\n Fallback (raw weather):")
    print(f"{city}: {weather}")


📌 Fallback (raw weather):
Bhopal: Weather data not available.
